# Analysis of Duchess' Eating Habits
William Reames

## Imports

In [1]:
import pandas as pd
import numpy as np

## Variables Used in During Data Analysis

In [2]:
minimum_satisfactory_immediate_eaten = 0.40
minimum_satisfactory_total_eaten = 0.80
num_previous_meals_to_check = 6  # This is used when checking how many times Duchess was given the same meal
pd.options.display.max_rows = None
pd.options.display.max_columns = None

## Importing the Dataset

In [3]:
df = pd.read_csv('../data/duchess-eating-habits.csv')

In [4]:
df.head()

,Date,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,Immediate % Eaten,Total % Eaten,Notes
0,10/25/2023,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,5%,10%,NaN
1,10/25/2023,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,60%,100%,NaN
2,10/26/2023,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,25%,80%,NaN
3,10/26/2023,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,30%,95%,NaN
4,10/27/2023,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,15%,60%,NaN


## Cleaning the Data

### Dropping unecessary data

In [5]:
df = df[~df['Meal'].isnull()]

In [6]:
df = df.drop('Notes', axis=1)

In [7]:
df = df.drop('Date', axis=1)

In [8]:
df.head()

,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,Immediate % Eaten,Total % Eaten
0,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,5%,10%
1,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,60%,100%
2,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,25%,80%
3,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,30%,95%
4,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,15%,60%


In [9]:
df.index.size

53

### Converting % eaten data to integer values

Converting values

In [10]:
df['immediate_eaten'] = pd.to_numeric(df['Immediate % Eaten'].str.replace('%', '')) / 100
df = df.drop('Immediate % Eaten', axis=1)

In [11]:
df['total_eaten'] = pd.to_numeric(df['Total % Eaten'].str.replace('%', '')) / 100
df = df.drop('Total % Eaten', axis=1)

In [12]:
df.head()

,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,immediate_eaten,total_eaten
0,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,0.05,0.10
1,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,0.60,1.00
2,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,0.25,0.80
3,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,0.30,0.95
4,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,0.15,0.60


Updating NaN values to average values

In [13]:
average_immediate_eaten = round(df['immediate_eaten'].mean(), 2)
average_total_eaten = round(df['total_eaten'].mean(), 2)
df['immediate_eaten'] = df['immediate_eaten'].fillna(average_immediate_eaten)
df['total_eaten'] = df['total_eaten'].fillna(average_total_eaten)

print(f'Average immediate amount eaten: {average_immediate_eaten}')
print(f'Average total amount eaten:     {average_total_eaten}')

Average immediate amount eaten: 0.31
Average total amount eaten:     0.72


### Determining food satisfaction levels

In [14]:
df['satisfactory_immediate_eaten'] = df['immediate_eaten'] >= minimum_satisfactory_immediate_eaten
df['satisfactory_total_eaten'] = df['total_eaten'] >= minimum_satisfactory_total_eaten

In [15]:
df.head()

,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,immediate_eaten,total_eaten,satisfactory_immediate_eaten,satisfactory_total_eaten
0,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,0.05,0.10,False,False
1,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,0.60,1.00,True,True
2,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,0.25,0.80,False,True
3,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,0.30,0.95,False,True
4,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,0.15,0.60,False,False


### Adding relevant data to each row based on previous rows

Adding a column for the previous meal's total_eaten value

In [16]:
for i in range(df.index.size):
    if i > 0 and df.loc[i - 1, 'total_eaten']:
        df.loc[i, 'previous_total_eaten'] = df.loc[i - 1, 'total_eaten']
    else:
        df.loc[i, 'previous_total_eaten'] = average_total_eaten

In [17]:
df.head()

,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,immediate_eaten,total_eaten,satisfactory_immediate_eaten,satisfactory_total_eaten,previous_total_eaten
0,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,0.05,0.10,False,False,0.72
1,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,0.60,1.00,True,True,0.10
2,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,0.25,0.80,False,True,1.00
3,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,0.30,0.95,False,True,0.80
4,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,0.15,0.60,False,False,0.95


Adding a column for the number of times that flavor was served during the previous 6 meals

In [18]:
for i in range(df.index.size):
    if i == 0:
        df.loc[i, 'num_same_previous_meals'] = 0
        continue
    min_index = i - num_previous_meals_to_check if i > num_previous_meals_to_check else 0
    max_index = i - 1 if i > 1 else 0
    meal_count = list(df.loc[min_index:max_index, 'Flavor']).count(df.loc[i, 'Flavor'])
    df.loc[i, 'num_same_previous_meals'] = meal_count

In [19]:
df.head()

,Meal,Brand,Flavor,Type,Initial Temperature,Microwaved,Added Water,immediate_eaten,total_eaten,satisfactory_immediate_eaten,satisfactory_total_eaten,previous_total_eaten,num_same_previous_meals
0,Lunch,Friskies,Salmon & Gravy,Chunky extra gravy,Cold,False,False,0.05,0.10,False,False,0.72,0.0
1,Dinner,Friskies,Turkey & Cheese,Shreds,Room,False,False,0.60,1.00,True,True,0.10,0.0
2,Lunch,Friskies,Turkey & Cheese,Shreds,Cold,False,False,0.25,0.80,False,True,1.00,1.0
3,Dinner,Friskies,Whitefish & Sardines,Shreds,Room,False,False,0.30,0.95,False,True,0.80,0.0
4,Lunch,Friskies,Whitefish & Sardines,Shreds,Cold,False,False,0.15,0.60,False,False,0.95,1.0


### Creating binary encodings

Lunch vs Dinner

In [20]:
df['lunch'] = pd.get_dummies(df['Meal'])['Lunch']
df = df.drop('Meal', axis=1)

Room vs Cold initial temperature

In [21]:
df['initially_cold'] = pd.get_dummies(df['Initial Temperature'])['Cold']
df = df.drop('Initial Temperature', axis=1)

In [22]:
df.head()

,Brand,Flavor,Type,Microwaved,Added Water,immediate_eaten,total_eaten,satisfactory_immediate_eaten,satisfactory_total_eaten,previous_total_eaten,num_same_previous_meals,lunch,initially_cold
0,Friskies,Salmon & Gravy,Chunky extra gravy,False,False,0.05,0.10,False,False,0.72,0.0,1,1
1,Friskies,Turkey & Cheese,Shreds,False,False,0.60,1.00,True,True,0.10,0.0,0,0
2,Friskies,Turkey & Cheese,Shreds,False,False,0.25,0.80,False,True,1.00,1.0,1,1
3,Friskies,Whitefish & Sardines,Shreds,False,False,0.30,0.95,False,True,0.80,0.0,0,0
4,Friskies,Whitefish & Sardines,Shreds,False,False,0.15,0.60,False,False,0.95,1.0,1,1


### Creating one-hot encodings

Flavors

In [23]:
df['Flavor'] = df['Flavor'].str.lower()
df['Flavor'] = df['Flavor'].str.replace('ocean whitefish', 'whitefish')
df['Flavor'] = df['Flavor'].str.replace('grain free salmon', 'salmon')

In [24]:
df[['flavor_1', 'flavor_2']] = df['Flavor'].str.split(' & ', expand=True)
df = df.drop('Flavor', axis=1)

In [25]:
df = pd.concat([df, pd.get_dummies(df['flavor_1'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['flavor_2'])], axis=1)
df = df.drop('flavor_1', axis=1)
df = df.drop('flavor_2', axis=1)

Brand

In [26]:
df = pd.concat([df, pd.get_dummies(df['Brand'].str.lower())], axis=1)
df = df.drop('Brand', axis=1)

Food type

In [27]:
df = pd.concat([df, pd.get_dummies(df['Type'].str.lower())], axis=1)
df = df.drop('Type', axis=1)

In [31]:
df.head()

,Microwaved,Added Water,immediate_eaten,total_eaten,satisfactory_immediate_eaten,satisfactory_total_eaten,previous_total_eaten,num_same_previous_meals,lunch,initially_cold,chicken,country style,mariner's catch,mixed grill,salmon,turkey,whitefish,cheese,giblets,gravy,sardines,tuna,blue wilderness,friskies,open nature,chunky extra gravy,meaty bits,pate,prime filets,shreds
0,False,False,0.05,0.10,False,False,0.72,0.0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0
1,False,False,0.60,1.00,True,True,0.10,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1
2,False,False,0.25,0.80,False,True,1.00,1.0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1
3,False,False,0.30,0.95,False,True,0.80,0.0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1
4,False,False,0.15,0.60,False,False,0.95,1.0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1


### Dropping immediate_eaten and total_eaten

In [33]:
df = df.drop('immediate_eaten', axis=1)
df = df.drop('total_eaten', axis=1)

In [38]:
df

,Microwaved,Added Water,satisfactory_immediate_eaten,satisfactory_total_eaten,previous_total_eaten,num_same_previous_meals,lunch,initially_cold,chicken,country style,mariner's catch,mixed grill,salmon,turkey,whitefish,cheese,giblets,gravy,sardines,tuna,blue wilderness,friskies,open nature,chunky extra gravy,meaty bits,pate,prime filets,shreds
0,False,False,False,False,0.72,0.0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0
1,False,False,True,True,0.10,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1
2,False,False,False,True,1.00,1.0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1
3,False,False,False,True,0.80,0.0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1
4,False,False,False,False,0.95,1.0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1
5,False,False,False,True,0.60,0.0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0
6,False,False,False,False,0.85,1.0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0
7,True,False,True,True,0.55,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
8,True,False,True,True,1.00,1.0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
9,False,False,True,True,0.95,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1


## Analyzing the Data

In [29]:
# TODO: separate satisfactory_*_eaten into Y columns for data analysis